In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score, roc_curve, auc

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bp-v1(pakai).xlsx to bp-v1(pakai).xlsx


In [ ]:
df = pd.read_excel("bp-v1(pakai).xlsx", index_col = None)
df

,tweet,sentimen
0,[blackpink] ayo blinks suara nya buat jisoo ðŸ...,1
1,ðŸ“¸ Lisa x ADLV Â @BLACKPINK https://t.co/Di...,1
2,"sabar ya blinks, kamu baca deh wawancara merek...",1
3,blinks gimana perasaan kalian akhir-akhir ini ...,1
4,Omong omong blinks aja ga nyangka jennie mengu...,1
...,...,...
595,YGE lu manusia bukan sih??! Cepat kembali in B...,0
596,Ini tau berita dari mana dah benaran di tunda ...,0
597,mau itu gaya YGE atau apa masalah kembali tap...,0
598,"Kasian blackpink udah cantik, sukses tapi agen...",0


In [ ]:
df["sentimen"].value_counts()

0    306
1    294
Name: sentimen, dtype: int64

In [ ]:
#menghilangkan url
df["tweet"] = df["tweet"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,tweet,sentimen
0,[blackpink] ayo blinks suara nya buat jisoo ðŸ...,1
1,ðŸ“¸ Lisa x ADLV Â @BLACKPINK,1
2,"sabar ya blinks, kamu baca deh wawancara merek...",1
3,blinks gimana perasaan kalian akhir-akhir ini ...,1
4,Omong omong blinks aja ga nyangka jennie mengu...,1
...,...,...
595,YGE lu manusia bukan sih??! Cepat kembali in B...,0
596,Ini tau berita dari mana dah benaran di tunda ...,0
597,mau itu gaya YGE atau apa masalah kembali tap...,0
598,"Kasian blackpink udah cantik, sukses tapi agen...",0


In [ ]:
#remove mention
import re
def remove_mention(text):
    for i in text:
        result = re.sub("@[A-Za-z0-9_]+","",text)
    return result

df["tweet"]=df["tweet"].apply(remove_mention)
df

,tweet,sentimen
0,[blackpink] ayo blinks suara nya buat jisoo ðŸ...,1
1,ðŸ“¸ Lisa x ADLV Â,1
2,"sabar ya blinks, kamu baca deh wawancara merek...",1
3,blinks gimana perasaan kalian akhir-akhir ini ...,1
4,Omong omong blinks aja ga nyangka jennie mengu...,1
...,...,...
595,YGE lu manusia bukan sih??! Cepat kembali in B...,0
596,Ini tau berita dari mana dah benaran di tunda ...,0
597,mau itu gaya YGE atau apa masalah kembali tap...,0
598,"Kasian blackpink udah cantik, sukses tapi agen...",0


In [ ]:
#remove hashtags
def remove_hash(text):
    for i in text:
        result = re.sub("#[A-Za-z0-9_]+","",text)
    return result

df["tweet"]=df["tweet"].apply(remove_hash)
df

,tweet,sentimen
0,[blackpink] ayo blinks suara nya buat jisoo ðŸ...,1
1,ðŸ“¸ Lisa x ADLV Â,1
2,"sabar ya blinks, kamu baca deh wawancara merek...",1
3,blinks gimana perasaan kalian akhir-akhir ini ...,1
4,Omong omong blinks aja ga nyangka jennie mengu...,1
...,...,...
595,YGE lu manusia bukan sih??! Cepat kembali in B...,0
596,Ini tau berita dari mana dah benaran di tunda ...,0
597,mau itu gaya YGE atau apa masalah kembali tap...,0
598,"Kasian blackpink udah cantik, sukses tapi agen...",0


In [ ]:
#menghilangkan punctuation
df["tweet"] = df["tweet"].str.replace(r'[^\w\s]+', '')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,tweet,sentimen
0,blackpink ayo blinks suara nya buat jisoo ðŸðŸ,1
1,ðŸ Lisa x ADLV Â,1
2,sabar ya blinks kamu baca deh wawancara mereka...,1
3,blinks gimana perasaan kalian akhirakhir ini s...,1
4,Omong omong blinks aja ga nyangka jennie mengu...,1
...,...,...
595,YGE lu manusia bukan sih Cepat kembali in Blac...,0
596,Ini tau berita dari mana dah benaran di tunda ...,0
597,mau itu gaya YGE atau apa masalah kembali tap...,0
598,Kasian blackpink udah cantik sukses tapi agens...,0


In [ ]:
#remove emoji
df["tweet"] = df["tweet"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df

,tweet,sentimen
0,blackpink ayo blinks suara nya buat jisoo,1
1,Lisa x ADLV,1
2,sabar ya blinks kamu baca deh wawancara mereka...,1
3,blinks gimana perasaan kalian akhirakhir ini s...,1
4,Omong omong blinks aja ga nyangka jennie mengu...,1
...,...,...
595,YGE lu manusia bukan sih Cepat kembali in Blac...,0
596,Ini tau berita dari mana dah benaran di tunda ...,0
597,mau itu gaya YGE atau apa masalah kembali tap...,0
598,Kasian blackpink udah cantik sukses tapi agens...,0


In [ ]:
#case folding
data_lower = []
for line in df["tweet"]:
  a = line.lower()
  data_lower.append(a) 

data_lower

['blackpink ayo blinks suara nya buat jisoo ',
 ' lisa x adlv  ',
 'sabar ya blinks kamu baca deh wawancara mereka sedih bgt pasti capek jg jd mereka  pasti mereka jg bakal kembali kok kita tunggu aja okey pasti anakanak paud bakal kasih yg terbaik buat kita  semangat blinks',
 'blinks gimana perasaan kalian akhirakhir ini sibuk sama anak blackpink  pengirim senang bgt ga nyangka bisa sesibuk ini promosi sensasional mereka seru banget akhirnya penantian kita blinks ',
 'omong omong blinks aja ga nyangka jennie mengunggah rollingstone kemarin soalnya foto sampul majalah dia belum keluar yg baru keluar baru lisa dan rose',
 'sesayang itu jujur dan jujur kangen bgt sama senyum indahnya nini  kamu berhak untuk bahagia sayang  tolong bahagia lah bahagia dgn semua pilihan kamu dgn semua yang kamu suka blinks akan selalu bahagia jika kamu bahagia juga blinks akan selalu mendukung pilihan kalian cinta kalian ',
 'awal gw suka  gegara jennie sama lisa muncul di tiktok gw suka jennie lisa makin 

In [ ]:
#remove number
import re
data_no_number = []
for line in data_lower:
  result = re.sub(r'\d', '', line)
  data_no_number.append(result)

data_no_number

['blackpink ayo blinks suara nya buat jisoo ',
 ' lisa x adlv  ',
 'sabar ya blinks kamu baca deh wawancara mereka sedih bgt pasti capek jg jd mereka  pasti mereka jg bakal kembali kok kita tunggu aja okey pasti anakanak paud bakal kasih yg terbaik buat kita  semangat blinks',
 'blinks gimana perasaan kalian akhirakhir ini sibuk sama anak blackpink  pengirim senang bgt ga nyangka bisa sesibuk ini promosi sensasional mereka seru banget akhirnya penantian kita blinks ',
 'omong omong blinks aja ga nyangka jennie mengunggah rollingstone kemarin soalnya foto sampul majalah dia belum keluar yg baru keluar baru lisa dan rose',
 'sesayang itu jujur dan jujur kangen bgt sama senyum indahnya nini  kamu berhak untuk bahagia sayang  tolong bahagia lah bahagia dgn semua pilihan kamu dgn semua yang kamu suka blinks akan selalu bahagia jika kamu bahagia juga blinks akan selalu mendukung pilihan kalian cinta kalian ',
 'awal gw suka  gegara jennie sama lisa muncul di tiktok gw suka jennie lisa makin 

In [ ]:
#remove whitespace
data_no_whitespace = []
for line in data_no_number:
  if len(line.split()) > 1:
    result = ' '.join(line.split())
    data_no_whitespace.append(result) 

data_no_whitespace

['blackpink ayo blinks suara nya buat jisoo',
 'lisa x adlv',
 'sabar ya blinks kamu baca deh wawancara mereka sedih bgt pasti capek jg jd mereka pasti mereka jg bakal kembali kok kita tunggu aja okey pasti anakanak paud bakal kasih yg terbaik buat kita semangat blinks',
 'blinks gimana perasaan kalian akhirakhir ini sibuk sama anak blackpink pengirim senang bgt ga nyangka bisa sesibuk ini promosi sensasional mereka seru banget akhirnya penantian kita blinks',
 'omong omong blinks aja ga nyangka jennie mengunggah rollingstone kemarin soalnya foto sampul majalah dia belum keluar yg baru keluar baru lisa dan rose',
 'sesayang itu jujur dan jujur kangen bgt sama senyum indahnya nini kamu berhak untuk bahagia sayang tolong bahagia lah bahagia dgn semua pilihan kamu dgn semua yang kamu suka blinks akan selalu bahagia jika kamu bahagia juga blinks akan selalu mendukung pilihan kalian cinta kalian',
 'awal gw suka gegara jennie sama lisa muncul di tiktok gw suka jennie lisa makin lama gw cari

In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 6.7 MB/s 


In [ ]:
#stemming untuk teks berbahasa Indonesia
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

translator = str.maketrans("", "", string.punctuation)
data_stemmed = map(lambda x: stemmer.stem(x), data_no_whitespace)
data_no_punc = map(lambda x: x.lower().translate(translator), data_stemmed)

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopword = stopwords.words("indonesian")
stopword.extend(["blackpink", "x", "deh", "bgt", "jg", "jd", "kok", "okey", "ga", "btw", "blm", "yg", "lu", "gw", "stalk", "cz", "kena", "woy", "anjir", "kek", "usah", "ya",
                 "goblok", "nya", "asu", "lah", "nggak", "anjrit", "jg", "gatau", "nder", "huhu", "sih", "ah", "sok", "duuh", "hp", "lhooo", "hihi", "mah", "kalo",
                 "dll", "yuk", "doang","rep", "banget", "lho", "gk", "gue", "misuh", "lol", "caper", "anjim", "bangsad", "tuh", "hello", "yuk", "si", "plis", "please", 
                 "gila", "nih", "kah", "eh", "guys", "lo", "gua", "pun", "&amp", "yah", "dong", "kalo", "kok", "an", "ayuk", "ayo", "yuk", "si", "hah", "dpt", "an", 
                 "pun", "kan", "hi", "heh", "hahaha", "in", "hiks", "bestie", "nge", "gpp", "eh", "an", "woi", "dah", "gini", "amat", "jt", "gini", "pls", "huhu",
                 "cuy", "woi", "cuy", "wkwk", "dll", "wib", "mampus", "gini", "allah", "alhamdulillah", "cenayang", "duh", "sumpah", "badas", "mba",
                 "noh", "fuck", "dih", "mas", "tuan", "pak", "nyonya", "rt", "k", "yuhuu", "up", "se", "trimz", "ku", "loh","ku", "njing", "astagfirullah", 
                 "oy", "ges", "bego", "u", "m", "ye", "as", "gak", "sialan", "njir", "kali", "anjing", "ber", "rd", "abis", "about", "adik", "adek", "adikadikmu",
                 "yok", "you", "your", "yukk", "aduh", "yaudah", "yeay", "yaa", "yak", "kau", "gaada", "iya", "benerbener", "no", "biarin", "oh", "gabisa", "bett", "haha",
                 "ni", "hehe", "dg", "so", "ngga", "tp", "anjirgue", "ntar", "kagak", "kesalkalo", "gapapa", "gegara", "ok", "okay", "iyaa", "my", "udahan", "bgtkenapa", 
                 "tau", "neng", "gajadi", "gapernah", "jgn", "lbh", "aja", "udah", "let", "kalo", "aja", "is", "bilang", "lihat", "tolong", "gitu", "situ", "lari", "tangan", "pas", "cari", "biar", "oke", "on",
                 "pakai", "ayah", "pas", "juta", "jam", "menit", "orang", "ubah", "papa", "by", "lupa", "mari", "duluan", "betapa", "ubah", "to", "anak", "pilih", "selamat", "ulang", "tahun",
                 "bikin", "gimana", "gin", "moga", "terima", "jual", "kirim", "capai", "lampau", "kayak", "langsung", "harga", "emang", "yey", "yuhu", "ygegua", "yt", "with"])

datafinal = []
for line in data_no_punc:
  word_token = nltk.word_tokenize(line)
  word_token = [word for word in word_token if word not in stopword and not word[0].isdigit()]
  datafinal.append(" ".join(word_token))

datafinal

['blinks suara jisoo',
 'lisa adlv',
 'sabar blinks baca wawancara sedih capek tunggu anakanak paud kasih semangat blinks',
 'blinks asa akhirakhir sibuk senang nyangka sibuk promosi sensasional seru blinks',
 'omong omong blinks nyangka jennie unggah rollingstone kemarin foto sampul majalah lisa rose',
 'sayang jujur jujur kangen senyum indah nini hak bahagia sayang bahagia bahagia dgn dgn suka blinks bahagia bahagia blinks dukung cinta',
 'suka jennie lisa muncul tiktok suka jennie lisa blinks kaget rumor kencan jennie kemarin jennie bayang jennie semangat jen blinks',
 'hebat blinks bangga jennie taehyung sehat lancar karirnya army benci jennie terimakasih keren',
 'berterimakasih blinks rollingstone',
 'blinks kaget batin tibatiba konten',
 'kagum pikir masingmasing member benarbenar sayang hidup lepas blinks',
 'halo blinks hasil seni pacar kemarin versi rose',
 'mutual blinks suka ngomong ngomong foto profil rosie',
 'bantu akun album cantumin nama terimakasih',
 'blinks rambut r

In [ ]:
#count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorize = vectorizer.fit_transform(datafinal)
vectorize = vectorize.toarray()
feature_vectorize = vectorizer.get_feature_names()
count_vector = pd.DataFrame(vectorize, columns=feature_vectorize)
count_vector.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,acara,acm,adil,adlv,aduk,aespa,agens,agustus,ajaib,ajak,...,winnerblackpink,wisata,wisuda,wkkw,wujud,yedam,yge,youth,youtube,zefanya
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform = TfidfTransformer(use_idf=True)
tfidf_ = tfidf_transform.fit_transform(vectorize)
tfidf = pd.DataFrame(tfidf_.A, columns=vectorizer.get_feature_names())
tfidf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,acara,acm,adil,adlv,aduk,aespa,agens,agustus,ajaib,ajak,...,winnerblackpink,wisata,wisuda,wkkw,wujud,yedam,yge,youth,youtube,zefanya
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.865978,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.237873,0.0,0.0,0.0
596,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
597,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.148971,0.0,0.0,0.0
598,0.0,0.0,0.0,0.000000,0.0,0.0,0.484387,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.208338,0.0,0.0,0.0


In [ ]:
x = tfidf_
y = df["sentimen"]

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
i = 1
for train_index, test_index in skf.split(x,y):
  print("Fold : ", i)
  print("TRAIN:", train_index, "\nTEST:", test_index)
  x_train, x_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]
  i += 1
  model = BernoulliNB()
  model.fit(x_train, y_train)
  # print("---------------------")
  # y_pred_prob_train = model.predict_proba(x_train)
  # print("NB classification probability for training data : \n\n", y_pred_prob_train)
  print("---------------------")
  y_pred_train = model.predict(x_train)
  cm_train = confusion_matrix(y_train, y_pred_train)
  print("Train Confusion matrix\n\n", cm_train)
  print("---------------------")
  TP_train = cm_train[0,0]
  TN_train = cm_train[1,1]
  FP_train = cm_train[0,1]
  FN_train = cm_train[1,0]
  accuracy_train = (TP_train+TN_train)/float(TP_train+TN_train+FP_train+FN_train)
  print("Train Accuracy : {0:0.4f}".format(accuracy_train))
  print("---------------------")
  AUC_train = roc_auc_score(y_train, y_pred_train)
  print("Train AUC : {:.4f}".format(AUC_train))
  print("---------------------")
  y_pred_prob_test = model.predict_proba(x_test)
  print("NB classification probability for testing data : \n\n", y_pred_prob_test)
  print("---------------------")
  y_pred_test = model.predict(x_test)
  cm_test = confusion_matrix(y_test, y_pred_test)
  print("---------------------")
  print("Test Confusion matrix\n\n", cm_test)
  print("---------------------")
  TP_test = cm_test[0,0]
  TN_test = cm_test[1,1]
  FP_test = cm_test[0,1]
  FN_test = cm_test[1,0]
  accuracy_test = (TP_test+TN_test)/float(TP_test+TN_test+FP_test+FN_test)
  print("Test Accuracy : {0:0.4f}".format(accuracy_test))
  print("---------------------")
  AUC_test = roc_auc_score(y_test, y_pred_test)
  print("Test AUC : {:.4f}".format(AUC_test))

Fold :  1
TRAIN: [  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  43  44  45  46  47  48  50  52  53  54  55  57  58  59
  60  61  63  64  65  66  67  68  69  71  72  73  74  77  78  79  80  82
  83  84  85  86  87  88  89  90  91  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 130 131 132 133 134 135 136 137 139
 140 141 142 143 144 145 146 147 148 149 150 152 154 155 156 157 158 159
 160 161 162 163 164 165 167 168 169 171 172 173 174 175 176 177 178 179
 180 181 182 184 185 187 189 190 191 192 193 194 195 196 197 198 199 201
 202 203 204 205 206 207 208 209 210 212 213 214 215 216 217 218 219 220
 221 223 224 225 226 227 228 229 230 231 232 233 234 235 237 238 239 240
 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258
 259 260 262 263 264 265 266 267 2

In [ ]:
sm = SMOTE()
x_res, y_res = sm.fit_resample(x, y)
x_train_res, x_test_res, y_train_res, y_test_res = train_test_split(x_res, y_res, test_size=0.25, random_state=42)
model = BernoulliNB()
model.fit(x_train_res, y_train_res)
y_pred_res_train = model.predict(x_train_res)
print("Train Accuracy: {:.2f}%".format(accuracy_score(y_train_res, y_pred_res_train) * 100))
print("Train AUC Score: {:.2f}%".format(roc_auc_score(y_train_res, y_pred_res_train) * 100))
print("\nTrain Confusion Matrix:\n", confusion_matrix(y_train_res, y_pred_res_train))
print("---------------------")
y_pred_prob_test = model.predict_proba(x_test_res)
y_pred_res_test = model.predict(x_test_res)
print("NB classification probability for testing data : \n\n", y_pred_prob_test)
print("Testing Accuracy: {:.2f}%".format(accuracy_score(y_test_res, y_pred_res_test) * 100))
print("Testing AUC Score: {:.2f}%".format(roc_auc_score(y_test_res, y_pred_res_test) * 100))
print("\nTesting Confusion Matrix:\n", confusion_matrix(y_test_res, y_pred_res_test))

Train Accuracy: 88.45%
Train AUC Score: 88.60%

Train Confusion Matrix:
 [[203  36]
 [ 17 203]]
---------------------
NB classification probability for testing data : 

 [[1.39036730e-01 8.60963270e-01]
 [9.66992948e-01 3.30070524e-02]
 [5.67712124e-04 9.99432288e-01]
 [2.26325578e-04 9.99773674e-01]
 [1.35349511e-01 8.64650489e-01]
 [2.64899555e-02 9.73510044e-01]
 [4.94524400e-01 5.05475600e-01]
 [2.00308515e-01 7.99691485e-01]
 [9.82278634e-01 1.77213662e-02]
 [2.86475357e-02 9.71352464e-01]
 [9.83141392e-01 1.68586085e-02]
 [3.02227327e-01 6.97772673e-01]
 [5.79056369e-01 4.20943631e-01]
 [9.35031975e-01 6.49680248e-02]
 [4.24545112e-01 5.75454888e-01]
 [9.98783143e-01 1.21685728e-03]
 [7.98031041e-01 2.01968959e-01]
 [7.00479820e-01 2.99520180e-01]
 [1.24480052e-01 8.75519948e-01]
 [6.24787315e-02 9.37521268e-01]
 [3.32002288e-02 9.66799771e-01]
 [1.70582960e-01 8.29417040e-01]
 [8.87709960e-01 1.12290040e-01]
 [4.58604941e-01 5.41395059e-01]
 [4.13679800e-02 9.58632020e-01]
 [9.6